In [2]:
# from PIL import Image
# import matplotlib.pyplot as plt
# import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix

In [3]:
batch_size = 1
epochs = 100
IMG_HEIGHT = 16
IMG_WIDTH = 16
IMG_CHANNELS = 3
color_mode = 'rgb'
CLASS_NAMES=np.array(['makise', 'cc', 'rem'])

In [4]:
train_dir = f"../../Dataset/{IMG_HEIGHT}x{IMG_WIDTH}/classic/train"
test_dir = f"../../Dataset/{IMG_HEIGHT}x{IMG_WIDTH}/classic/test"

In [5]:
# Generator for our training data
train_image_generator = ImageDataGenerator(rotation_range=30, 
                                           width_shift_range=0.3, 
                                           height_shift_range=0.3, 
                                           brightness_range=(0.5, 1.5), 
                                           zoom_range=0.3, 
                                           horizontal_flip=True, 
                                           rescale=1./255)
# Generator for our validation data
validation_image_generator = ImageDataGenerator(rotation_range=30, 
                                           width_shift_range=0.3, 
                                           height_shift_range=0.3, 
                                           brightness_range=(0.5, 1.5), 
                                           zoom_range=0.3, 
                                           horizontal_flip=True, 
                                           rescale=1./255) 

In [6]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           color_mode=color_mode,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           classes=list(CLASS_NAMES),
                                                           class_mode='categorical')

Found 1043 images belonging to 3 classes.


In [7]:
validation_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=test_dir,
                                                           shuffle=True,
                                                           color_mode=color_mode,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           classes=list(CLASS_NAMES),
                                                           class_mode='categorical')

Found 180 images belonging to 3 classes.


In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)),
    keras.layers.Dense(1000, activation='tanh'),
    keras.layers.Dense(100, activation='tanh'),
    keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.mse,
              metrics=['categorical_accuracy'])

In [ ]:
log_dir = f"Tests/Keras/{IMG_HEIGHT}x{IMG_WIDTH}-tanh-{IMG_WIDTH*IMG_HEIGHT*IMG_CHANNELS}-1000-100-001-sgd-mse"
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
log_dir

'Tests/Keras/64x64-tanh-12288-1000-100-001-sgd-mse'

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              12289000  
_________________________________________________________________
dense_1 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 12,389,403
Trainable params: 12,389,403
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=validation_data_gen,
    callbacks=[tb_callback]
)

Epoch 1/100
1043/1043 [==============================] - 33s 32ms/step - loss: 0.2108 - categorical_accuracy: 0.4746 - val_loss: 0.1717 - val_categorical_accuracy: 0.6222
Epoch 2/100
1043/1043 [==============================] - 44s 42ms/step - loss: 0.1803 - categorical_accuracy: 0.5570 - val_loss: 0.1608 - val_categorical_accuracy: 0.6667
Epoch 3/100
1043/1043 [==============================] - 38s 36ms/step - loss: 0.1672 - categorical_accuracy: 0.6222 - val_loss: 0.1797 - val_categorical_accuracy: 0.5889
Epoch 4/100
1043/1043 [==============================] - 34s 33ms/step - loss: 0.1593 - categorical_accuracy: 0.6443 - val_loss: 0.1419 - val_categorical_accuracy: 0.7111
Epoch 5/100
1043/1043 [==============================] - 32s 31ms/step - loss: 0.1527 - categorical_accuracy: 0.6817 - val_loss: 0.1327 - val_categorical_accuracy: 0.7278
Epoch 6/100
1043/1043 [==============================] - 32s 30ms/step - loss: 0.1519 - categorical_accuracy: 0.6779 - val_loss: 0.1460 - val_cat

In [ ]:
keras_y_preds_train = model.predict(x_train)
keras_y_preds_val = model.predict(x_val)

In [ ]:
keras_cfmat_train = confusion_matrix(y_train, keras_y_preds_train)
keras_cfmat_val = confusion_matrix(y_val, keras_y_preds_val)

In [8]:
from mlp_py import PyMLP

In [9]:
def data_generator(train_data_gen, validation_data_gen, nb_img_train, nb_img_val):
  x_train = []
  y_train = []
  x_val = []
  y_val = []
  for i in range(nb_img_train):
    image_batch, label_batch = next(train_data_gen)
    x_train.append(image_batch[0].flatten())
    y_train.append(np.where(label_batch[0].flatten() == 1., 1., -1.))
    
    if i < nb_img_val:
        image_batch, label_batch = next(validation_data_gen)
        x_val.append(image_batch[0].flatten())
        y_val.append(np.where(label_batch[0].flatten() == 1., 1., -1.))
  x_train = np.array(x_train, copy=False)
  y_train = np.array(y_train, copy=False)
  x_val = np.array(x_val, copy=False)
  y_val = np.array(y_val, copy=False)
  return (x_train, y_train, x_val, y_val)

In [10]:
x_train, y_train, x_val, y_val = data_generator(train_data_gen, validation_data_gen, 3000, 600)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (3000, 768)
y_train:  (3000, 3)
x_val:  (600, 768)
y_val:  (600, 3)


In [11]:
layer_sizes = [x_train.shape[0], 256, 48, y_train.shape[0]]
lr = 0.001
mlp = PyMLP(layer_sizes, lr, True)

In [12]:
def train_model(model, xtrain, ytrain, xval, yval, epochs):
  metrics = []
  for i in range(epochs):
    metric = model.fit_epoch(xtrain, ytrain, xval, yval)
    metrics.append(metric.tolist())
  return np.array(metrics).astype('float64', copy=False)

In [ ]:
metrics = train_model(mlp, x_train, y_train, x_val, y_val, epochs)

In [ ]:
rust_y_preds_train = mlp.predict(x_train, converted=True)
rust_y_preds_val = mlp.predict(x_val, converted=True)

In [ ]:
train_loss = metrics[:, 0]
train_acc = metrics[:, 1]
val_loss = metrics[:, 0]
val_acc = metrics[:, 1]

In [ ]:
rust_cfmat_train = mlp.confusion_matrix(y_train, y_pred_train)
rust_cfmat_val = mlp.confusion_matrix(y_val, y_pred_val)

In [ ]:
weights = mlp.get_weight()
bias = mlp.get_bias()